# Tarea 3
## Supply Chain Analytics

### Maestría en Inteligencia Analítica para la Toma de Decisiones
#### Universidad de los Andes
#### Bogotá - 2020

### Replique el problema de localización y asignación visto en clase (programa no lineal) usando Pyomo.

# Cargue de librerías a usar

In [1]:
#Librerias par el procesamiento de los datos
import pandas as pd
import numpy as np
#Librerias para visuzalización gráfica
import matplotlib.pyplot as plt
#Libreria para el modelo de optimización
from pyomo.environ import *

# Modelado
## Definición de las variables del modelo

In [2]:
#Datos
#Variable para la cantidad máxima de fábricas a instalar
P = 4 
#Variable para las ciudades de las fábricas a instalar, también la usaremos como variable de la demanda
C = ['Atlanta', 'Boston', 'Charlotte', 'Chicago', 'Cincinnati', 'Detroit', 'New York', 'Pittsburgh', 'St. Louis']
#Demanda por ciudad
D = {'Atlanta':572000, 'Boston':610000, 'Charlotte':723000,\
    'Chicago':2870000, 'Cincinnati':333000, 'Detroit':901000,\
    'New York':8450000, 'Pittsburgh':306000, 'St. Louis':350000
}
#Distancia entre ciudades
d = {	('Atlanta','Atlanta'):0,\
	('Boston','Atlanta'):1099,\
	('Charlotte','Atlanta'):245,\
	('Chicago','Atlanta'):720,\
	('Cincinnati','Atlanta'):461,\
	('Detroit','Atlanta'):722,\
	('New York','Atlanta'):884,\
	('Pittsburgh','Atlanta'):685,\
	('St. Louis','Atlanta'):555,\
	('Atlanta','Boston'):1099,\
	('Boston','Boston'):0,\
	('Charlotte','Boston'):867,\
	('Chicago','Boston'):996,\
	('Cincinnati','Boston'):907,\
	('Detroit','Boston'):721,\
	('New York','Boston'):219,\
	('Pittsburgh','Boston'):589,\
	('St. Louis','Boston'):1217,\
	('Atlanta','Charlotte'):245,\
	('Boston','Charlotte'):867,\
	('Charlotte','Charlotte'):0,\
	('Chicago','Charlotte'):769,\
	('Cincinnati','Charlotte'):479,\
	('Detroit','Charlotte'):629,\
	('New York','Charlotte'):645,\
	('Pittsburgh','Charlotte'):448,\
	('St. Louis','Charlotte'):715,\
	('Atlanta','Chicago'):720,\
	('Boston','Chicago'):996,\
	('Charlotte','Chicago'):769,\
	('Chicago','Chicago'):0,\
	('Cincinnati','Chicago'):296,\
	('Detroit','Chicago'):283,\
	('New York','Chicago'):790,\
	('Pittsburgh','Chicago'):461,\
	('St. Louis','Chicago'):297,\
	('Atlanta','Cincinnati'):461,\
	('Boston','Cincinnati'):907,\
	('Charlotte','Cincinnati'):479,\
	('Chicago','Cincinnati'):296,\
	('Cincinnati','Cincinnati'):0,\
	('Detroit','Cincinnati'):263,\
	('New York','Cincinnati'):667,\
	('Pittsburgh','Cincinnati'):288,\
	('St. Louis','Cincinnati'):359,\
	('Atlanta','Detroit'):722,\
	('Boston','Detroit'):721,\
	('Charlotte','Detroit'):629,\
	('Chicago','Detroit'):283,\
	('Cincinnati','Detroit'):263,\
	('Detroit','Detroit'):0,\
	('New York','Detroit'):614,\
	('Pittsburgh','Detroit'):286,\
	('St. Louis','Detroit'):531,\
	('Atlanta','New York'):884,\
	('Boston','New York'):219,\
	('Charlotte','New York'):645,\
	('Chicago','New York'):790,\
	('Cincinnati','New York'):667,\
	('Detroit','New York'):614,\
	('New York','New York'):0,\
	('Pittsburgh','New York'):371,\
	('St. Louis','New York'):976,\
	('Atlanta','Pittsburgh'):685,\
	('Boston','Pittsburgh'):589,\
	('Charlotte','Pittsburgh'):448,\
	('Chicago','Pittsburgh'):461,\
	('Cincinnati','Pittsburgh'):288,\
	('Detroit','Pittsburgh'):286,\
	('New York','Pittsburgh'):371,\
	('Pittsburgh','Pittsburgh'):0,\
	('St. Louis','Pittsburgh'):602,\
	('Atlanta','St. Louis'):555,\
	('Boston','St. Louis'):1217,\
	('Charlotte','St. Louis'):715,\
	('Chicago','St. Louis'):297,\
	('Cincinnati','St. Louis'):359,\
	('Detroit','St. Louis'):531,\
	('New York','St. Louis'):976,\
	('Pittsburgh','St. Louis'):602,\
	('St. Louis','St. Louis'):0,
 }

In [3]:
#Creamos el modelo y definimos los conjuntos de fábricas y clientes
model = ConcreteModel()
model.Locations = C
model.Customers = C
#Creamos la variable x como una matriz binaria, compuesta entre fábricas y clientes
model.x = Var(model.Locations, model.Customers, bounds=(0.0,1.0))   
#Creamos la variable que me indica si debo abrir una fábrica ahí
model.y = Var(model.Locations, within = Binary)
#Definimos nuestra variable objetivo como la demanda ponderada por distancia de las fabricas abiertas,
#que atienden a determinados clientes
model.obj = Objective(expr = sum(d[n,m]*model.x[n,m]*D[m] for n in model.Locations for m in model.Customers))

In [4]:
#Definimos nuestras restricciones
#Restricciones para X
model.single_x = ConstraintList()
#Todos los clientes deben ser satisfechos por 1 SOLA fábrica
for m in model.Customers:
    model.single_x.add(sum( model.x[n,m] for n in model.Locations ) == 1.0)
model.bound_y = ConstraintList()
for n in model.Locations:
    for m in model.Customers:
        model.bound_y.add(model.x[n,m] <= model.y[n])
#Puedo abrir un máximo de P fábricas, en este caso, 4
model.num_facilities = Constraint(expr=sum( model.y[n] for n in model.Locations ) <= P)

In [5]:
#Creamos el solver como GLPK (GNU Linear Programming KIT)
solver = SolverFactory('glpk')
#Solucionamos el modelo como minimización de la función objetivo
solver.solve(model, tee=False)
print('╔═════════════════════╗')
print('║       Solución      ║')
print('╚═════════════════════╝')

for i in model.Locations:
    if model.y[i] == 1:
        print("La fábrica en ",i, "atiende:")
    else:
        continue
    for j in model.Customers:
        if model.x[i,j] != 0:
            print("-->", j, "-- {}%".format(value(model.x[i,j])*100))
    print()
print()
print('╔═════════════════════╗')
print('║   Función Objetivo  ║')
print('╚═════════════════════╝')
print("Valor de la función objetivo :%.0f" %value(model.obj))

print()
print('╔═════════════════════╗')
print('║ Detalle por fábrica ║')
print('╚═════════════════════╝')
for wl in C:
    if value(model.y[wl]) > 0.5:
        customers = [str(cl) for cl in C if value(model.x[wl, cl]) > 0.5]
        print(str(wl)+" Atiende los clientes de: "+str(customers))
    else:
        print(str(wl)+": NO se debe construir")

╔═════════════════════╗
║       Solución      ║
╚═════════════════════╝
La fábrica en  Charlotte atiende:
--> Atlanta -- 100.0%
--> Charlotte -- 100.0%

La fábrica en  Chicago atiende:
--> Chicago -- 100.0%
--> St. Louis -- 100.0%

La fábrica en  Detroit atiende:
--> Cincinnati -- 100.0%
--> Detroit -- 100.0%
--> Pittsburgh -- 100.0%

La fábrica en  New York atiende:
--> Boston -- 100.0%
--> New York -- 100.0%


╔═════════════════════╗
║   Función Objetivo  ║
╚═════════════════════╝
Valor de la función objetivo :552775000

╔═════════════════════╗
║ Detalle por fábrica ║
╚═════════════════════╝
Atlanta: NO se debe construir
Boston: NO se debe construir
Charlotte Atiende los clientes de: ['Atlanta', 'Charlotte']
Chicago Atiende los clientes de: ['Chicago', 'St. Louis']
Cincinnati: NO se debe construir
Detroit Atiende los clientes de: ['Cincinnati', 'Detroit', 'Pittsburgh']
New York Atiende los clientes de: ['Boston', 'New York']
Pittsburgh: NO se debe construir
St. Louis: NO se debe cons